# Product Metrics Dashboard

This notebook loads the synthetic dataset and performs EDA, computes KPIs, and creates plots. See README.md for file list.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

DF = pd.read_csv('product_metrics_events.csv', parse_dates=['date','reg_date'])
DF.head()

## Daily metrics (DAU, sessions, revenue, ARPU)

In [ ]:
daily = DF.groupby(DF['date'].dt.date).agg(dau=('user_id','nunique'), sessions=('sessions','sum'), revenue=('revenue','sum')).reset_index()
daily['date'] = pd.to_datetime(daily['date'])
daily['arpu'] = daily['revenue'] / daily['dau'].replace({0:pd.NA})
daily.head()

## Cohort retention (weekly cohorts)

In [ ]:
DF['reg_week'] = DF['reg_date'].dt.to_period('W').apply(lambda r: r.start_time.date())
DF['week'] = DF['date'].dt.to_period('W').apply(lambda r: r.start_time.date())
cohort = DF.groupby(['reg_week','week']).agg(active_users=('user_id','nunique')).reset_index()
cohort['cohort_week'] = ((pd.to_datetime(cohort['week']) - pd.to_datetime(cohort['reg_week'])).dt.days // 7).astype(int)
cohort_pivot = cohort.pivot_table(values='active_users', index='reg_week', columns='cohort_week', aggfunc='sum').fillna(0)
cohort_pivot.head()

## Plots (saved to files)

In [ ]:
import matplotlib.pyplot as plt

# DAU
plt.figure(figsize=(10,4))
plt.plot(daily['date'], daily['dau'])
plt.title('Daily Active Users (DAU)')
plt.show()

# ARPU
plt.figure(figsize=(10,4))
plt.plot(daily['date'], daily['arpu'])
plt.title('Daily ARPU')
plt.show()

## SQL queries
See `product_metrics_queries.sql` for ready-to-use SQL examples

## LTV by cohort
The following cells compute average cumulative revenue (LTV) per cohort-week and plot cohort LTV trends.

In [ ]:
# LTV by cohort
DF = pd.read_csv('product_metrics_events.csv', parse_dates=['date','reg_date'])
DF['reg_week'] = DF['reg_date'].dt.to_period('W').apply(lambda r: r.start_time.date())
DF['week'] = DF['date'].dt.to_period('W').apply(lambda r: r.start_time.date())
DF['cohort_week'] = ((pd.to_datetime(DF['week']) - pd.to_datetime(DF['reg_week'])).dt.days // 7).astype(int)

user_week = DF.groupby(['reg_week','cohort_week','user_id']).agg(user_revenue=('revenue','sum')).reset_index()
user_week = user_week.sort_values(['reg_week','user_id','cohort_week'])
user_week['cum_revenue'] = user_week.groupby(['reg_week','user_id'])['user_revenue'].cumsum()

ltv = user_week.groupby(['reg_week','cohort_week']).agg(users_in_cohort=('user_id','nunique'), avg_cum_revenue=('cum_revenue','mean')).reset_index()
ltv.head()

## RFM segmentation
Calculate Recency (days since last activity), Frequency (number of active days), Monetary (total revenue) per user. Then create R/F/M quintiles and basic segments.

In [ ]:
# RFM calculation
DF = pd.read_csv('product_metrics_events.csv', parse_dates=['date','reg_date'])
reference_date = DF['date'].max() + pd.Timedelta(days=1)

user_agg = DF.groupby('user_id').agg(last_date=('date','max'), active_days=('date','nunique'), total_revenue=('revenue','sum')).reset_index()
user_agg['recency_days'] = (reference_date - pd.to_datetime(user_agg['last_date'])).dt.days
user_agg['frequency'] = user_agg['active_days']
user_agg['monetary'] = user_agg['total_revenue']

user_agg['r_quintile'] = pd.qcut(user_agg['recency_days'].rank(method='first'), 5, labels=[5,4,3,2,1]).astype(int)
user_agg['f_quintile'] = pd.qcut(user_agg['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
user_agg['m_quintile'] = pd.qcut(user_agg['monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

user_agg['RFM_score'] = user_agg['r_quintile'].astype(str) + user_agg['f_quintile'].astype(str) + user_agg['m_quintile'].astype(str)
user_agg['RFM_numeric'] = user_agg['r_quintile']*100 + user_agg['f_quintile']*10 + user_agg['m_quintile']

def rfm_segment(x):
    if x >= 100 and x < 233:
        return 'Champions'
    if x >= 233 and x < 344:
        return 'Loyal'
    if x >= 344 and x < 455:
        return 'Potential'
    if x >= 455 and x < 566:
        return 'At Risk'
    return 'Hibernating'

user_agg['segment'] = user_agg['RFM_numeric'].apply(rfm_segment)
user_agg.head()